In [134]:
# Add module import path to system path.
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Hot-loading modules before executing code: allows us to write code while testing it in this notebook.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
import logging

# Logging strategy for this notebook.
root = logging.getLogger()
root.setLevel(logging.INFO)

if not root.hasHandlers():
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)

    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)

    root.addHandler(handler)

In [136]:
from parser import Parser

parser = Parser()

In [137]:
teams = parser.parse_teams()

2022-03-10 18:38:57,004 - parser - INFO - Column names are TeamID, TeamName, FirstD1Season, LastD1Season
2022-03-10 18:38:57,006 - parser - INFO - Processed 372 teams.


In [138]:
seasons, teams = parser.parse()

2022-03-10 18:38:57,464 - parser - INFO - Column names are TeamID, TeamName, FirstD1Season, LastD1Season
2022-03-10 18:38:57,466 - parser - INFO - Processed 372 teams.
2022-03-10 18:38:57,467 - parser - INFO - Column names are Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, NumOT
2022-03-10 18:38:57,987 - parser - INFO - Processed 174471 games.
2022-03-10 18:38:57,988 - parser - INFO - Column names are Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, NumOT
2022-03-10 18:38:57,994 - parser - INFO - Processed 2317 games.
2022-03-10 18:38:57,995 - parser - INFO - Column names are Season, DayZero, RegionW, RegionX, RegionY, RegionZ
2022-03-10 18:38:57,999 - parser - INFO - Processed 38 seasons.


In [139]:
train_seasons = []
for year in range(1985, 2016):
    train_seasons.append(seasons[year])

test_seasons = []
for year in range(2016, 2020):
    test_seasons.append(seasons[year])

In [140]:
from season import Span

train_span = Span(train_seasons)
test_span = Span(test_seasons)

In [141]:
classifier = train_span.train()

In [147]:
from prediction import FiftyFiftyClassifier

span_predictions = test_span.predict(classifier)

In [148]:
# Build UI that lets you pick the features to use. 

# Train n times and give average score + min/max score per year and total over the span of years used for validating.

In [149]:
Span.score(span_predictions)

{2016: 0.69122351722072,
 2017: 0.6930358246850573,
 2018: 0.6921296709528887,
 2019: 0.69122351722072,
 -1: 0.6919031325198465}